In [1]:
from IPython.display import HTML

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import zipfile
import logging
import fsspec
# import ujson
from glob import glob
import os

from azure.storage.blob import ContainerClient
import tempfile
from metpy import xarray

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pandas as pd

In [3]:
from dask.distributed import Client
client = Client(n_workers=8)
client

/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46153 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:42333 Dashboard: /proxy/8787/status,Cluster Workers: 8 Cores: 8 Memory: 32.00 GiB


### Do setup stuff to get file paths

In [4]:
%%time
fs = fsspec.filesystem("reference", fo="../ReferenceMaker/combined.json",
                       remote_protocol='az', remote_options=dict(account_name='goeseuwest'),
                       simple_templates=True)
m = fs.get_mapper("")
# urllist = [goes_blob_root + u + "#mode=bytes" for u in blobs]

CPU times: user 17.7 s, sys: 1.94 s, total: 19.7 s
Wall time: 34.8 s


In [5]:
%%time
ds = xr.open_dataset(m, engine='zarr')

CPU times: user 17.8 s, sys: 673 ms, total: 18.4 s
Wall time: 18.2 s


In [3]:
connection_string = os.getenv('azure_connection_string')

In [6]:
fs_out = fsspec.get_mapper('az://goes-zarr/goes_from_reference.zarr/', connection_string=connection_string)


In [ ]:
%%time
out = ds.to_zarr(fs_out, mode='w', compute=False)

In [ ]:
%%time
import dask
dask.compute(out)